In [7]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
from glob import glob
import xarray as xr
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from einops import rearrange, reduce
import matplotlib.pyplot as plt
import huggingface_hub as hf_hub
import datasets as ds
from PIL import Image
import torch
import tarfile

from torch.utils.data import DataLoader
# import transforms
from torchvision.transforms import Compose, ToTensor, Normalize
import requests

In [8]:
state = "haryana"
files = sorted(glob(f"/home/jaiswalsuraj/bkdb/india/{state}/*.zarr"))
print(len(files))

40997


In [9]:
os.system(f"rm -rf /tmp/hfbk_{state}")
os.system(f"mkdir /tmp/hfbk_{state}")

def process(file):
    with xr.open_zarr(file, consolidated=False) as ds:
        pass
    try:
        lat = ds['lat'].values.item()
    except KeyError:
        print(file, "lat not found")
        return
    try:
        lon = ds['lon'].values.item()
    except KeyError:
        print(file, "lon not found")
        return 
    lat_lon = f"{lat}_{lon}"
    val = ds['data'].values
    val = rearrange(val, 'rb cb r c ch -> (rb r) (cb c) ch')
    # da = xr.DataArray(val, dims=['lat_lon', 'r', 'c', 'ch'], coords={'lat_lon': [lat_lon]})
    # return da
    img = Image.fromarray(val)
    img.save(f"/tmp/hfbk_{state}/{file.split('/')[-1].replace('.zarr', '.png')}")

# da_list = Parallel(n_jobs=48)(delayed(process)(file) for file in tqdm(files[:2048]))
# ds = xr.concat(da_list, dim='lat_lon').to_dataset(name='data')
# ds.to_zarr(f"/tmp/hfbk_{state}/data.zarr.zip", mode='w')
_ = Parallel(n_jobs=48)(delayed(process)(file) for file in tqdm(files))

  0%|          | 0/40997 [00:00<?, ?it/s]

/home/jaiswalsuraj/bkdb/india/haryana/28.75,76.20.zarr lat not found
/home/jaiswalsuraj/bkdb/india/haryana/28.75,76.23.zarr lat not found
/home/jaiswalsuraj/bkdb/india/haryana/28.75,76.24.zarr lon not found
/home/jaiswalsuraj/bkdb/india/haryana/28.75,76.32.zarr lat not found
/home/jaiswalsuraj/bkdb/india/haryana/28.75,76.49.zarr lat not found
/home/jaiswalsuraj/bkdb/india/haryana/30.07,76.74.zarr lat not found
/home/jaiswalsuraj/bkdb/india/haryana/30.07,76.75.zarr lat not found
/home/jaiswalsuraj/bkdb/india/haryana/30.07,76.80.zarr lon not found
/home/jaiswalsuraj/bkdb/india/haryana/30.07,76.87.zarr lat not found
/home/jaiswalsuraj/bkdb/india/haryana/30.07,76.89.zarr lat not found
/home/jaiswalsuraj/bkdb/india/haryana/30.07,76.99.zarr lat not found
/home/jaiswalsuraj/bkdb/india/haryana/30.07,77.00.zarr lat not found


In [12]:
# create a tar file each containing b_size images
b_size = 4096*2
files = sorted(glob(f"/tmp/hfbk_{state}/*.png"))
print(len(files))

for idx, i in tqdm(enumerate(range(0, len(files), b_size))):
    name = f"/tmp/hfbk_{state}/data_{str(idx).zfill(3)}.tar"
    print("Writing", name)
    with tarfile.open(name, "w") as tar:
        for file in files[i:i+b_size]:
            tar.add(file, arcname=file.split('/')[-1])

40985


0it [00:00, ?it/s]

Writing /tmp/hfbk_haryana/data_000.tar
Writing /tmp/hfbk_haryana/data_001.tar
Writing /tmp/hfbk_haryana/data_002.tar
Writing /tmp/hfbk_haryana/data_003.tar
Writing /tmp/hfbk_haryana/data_004.tar
Writing /tmp/hfbk_haryana/data_005.tar


In [16]:
# save file names to metadata
df = pd.DataFrame(map(lambda x: x.split("/")[-1].rsplit(".", 1)[0], files), columns=['file'])
df['lat'] = df['file'].apply(lambda x: float(x.split(',')[0]))
df['lon'] = df['file'].apply(lambda x: float(x.split(',')[1]))
df.to_csv(f"/tmp/hfbk_{state}/metadata.csv", index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47477 entries, 0 to 47476
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   file    47477 non-null  object 
 1   lat     47477 non-null  float64
 2   lon     47477 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1.1+ MB


## Appendix

```py
import os

import datasets
from datasets.tasks import ImageClassification


_DATA_URL = {
    "train": [f"data/data_{i}.tar" for i in range(6)],
}


class Imagenet1k(datasets.GeneratorBasedBuilder):
    VERSION = datasets.Version("1.0.0")

    def _info(self):
        return datasets.DatasetInfo(
            features=datasets.Features(
                {
                    "image": datasets.Image(),
                    "name": datasets.Value("string"),
                }
            ),
            # task_templates=[ImageClassification(image_column="image", label_column="label")],
        )

    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        archives = dl_manager.download(_DATA_URL)

        return [
            datasets.SplitGenerator(
                name=datasets.Split.TRAIN,
                gen_kwargs={
                    "archives": [dl_manager.iter_archive(archive) for archive in archives["train"]],
                    "split": "train",
                },
            ),
        ]

    def _generate_examples(self, archives, split):
        """Yields examples."""
        idx = 0
        for archive in archives:
            for path, file in archive:
                if path.endswith(".png"):
                    name = path.split("/")[-1].rsplit(".", 1)[0]
                    ex = {"image": {"path": path, "bytes": file.read()}, "name": name}
                    yield idx, ex
                    idx += 1
```

In [9]:
from pytube import YouTube

yt = YouTube('https://www.youtube.com/watch?v=7NNxK3CqaDk')
stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
print(stream)
stream.download()

[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">]
